In [1]:
#Data Analysis
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#Data Preprocessing and Feature Engineering
from textblob import TextBlob
import re

# !pip install tweet-preprocessor
# !pip install textblob

import nltk
# nltk.download('stopwords')
# nltk.download('wordnet')
# nltk.download('punkt')

from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer



import preprocessor as p

#Model Selection and Validation
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, classification_report,accuracy_score

In [2]:
OBAMA = 'final-testData-no-label-Obama-tweets(1).xlsx'
ROMNEY = 'final-testData-no-label-Romney-tweets(1).xlsx'
df_obama = pd.read_excel(OBAMA, sheet_name=0, header=None,index_col=None, 
                   skiprows=[0,1], names=['tweet'])
df_romney = pd.read_excel(ROMNEY, sheet_name=0, header=None,index_col=None, 
                   skiprows=[0,1], names=['tweet'])

In [13]:
def clean_data(unclean_tweets):
    cleaned_data = []
    for tweet in unclean_tweets:
        tweet = tweet.lower()
        tweet = re.sub(r'@\w+', r'', tweet)
        tweet = re.sub('<[^<]+?>', '', tweet)
        tweet = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', tweet)
        tweet = re.sub(r'(\s)@\w+', r'', tweet)
        tweet = re.sub(r'[<>!#@$:.,%\?-]+', r'', tweet)
        tweet = tweet.replace("'", "").replace("\"","")
        words = nltk.word_tokenize(tweet.lower())
        cleaned_data.append(words)
    return cleaned_data

In [ ]:
# # #Cast the results column to string since it contains both 2 & '2'
# df_obama['class'] = df_obama['class'].astype(str)
# sns.countplot(x = 'class', data = df_obama)

# df_romney

In [ ]:
# df_romney['class'] = df_romney['class'].astype(str)
# sns.countplot(x = 'class', data = df_romney)

In [ ]:
# # Drop rows not having 0, 1, -1
# df_obama = df_obama[df_obama['class'].isin(['0', '1', '-1'])] 
# df_romney = df_romney[df_romney['class'].isin(['0', '1', '-1'])] 

# # Print the shape of the dataframe 
# print(df_obama.shape) 
# print(df_romney.shape) 

In [ ]:
# sns.countplot(x = 'class', data = df_obama)

In [ ]:
df_obama

In [3]:
df_obama['tweet'] = df_obama['tweet'].astype(str)
df_obama['tweet_length'] = df_obama['tweet'].apply(lambda x: len(str(x)))

df_romney['tweet'] = df_romney['tweet'].astype(str)
df_romney['tweet_length'] = df_romney['tweet'].apply(lambda x: len(str(x)))

In [4]:
def clean_sentence(tweet):
    tweet = re.sub(re.compile('<[^>]+>'), '', tweet)
    tweet_blob = TextBlob(tweet)
    return ' '.join(tweet_blob.words)

def clean_stopwords(tweet):
    tweet_list = [ele for ele in tweet.split() if ele != 'user']
    clean_tokens = [t for t in tweet_list if re.match(r'[^\W\d]*$', t)]
    clean_s = ' '.join(clean_tokens)
    clean_mess = [word for word in clean_s.split() if word.lower() not in stopwords.words('english')]
    return clean_mess

# lexical normalization
def normalization(tweet_list):
    lem = WordNetLemmatizer()
    normalized_tweet = []
    for word in tweet_list:
        normalized_text = lem.lemmatize(word,'v')
        normalized_tweet.append(normalized_text)
    return normalized_tweet

In [5]:
df_obama['tweet_cleaned'] = df_obama['tweet'].apply(lambda x: normalization(clean_stopwords(clean_sentence(x))))
df_obama.head(10)
df_romney['tweet_cleaned'] = df_romney['tweet'].apply(lambda x: normalization(clean_stopwords(clean_sentence(x))))
df_romney.head(10)

,tweet,tweet_length,tweet_cleaned
3,I actually like <e>Romney </e>'s response to ...,58,"[actually, like, Romney, response, immigration]"
4,Just for that <a>immigration statement </a>tha...,115,"[immigration, statement, Romney, answer, enoug..."
5,This man <e>Romney </e>is tearing this dude ...,68,"[man, Romney, tear, dude, economics]"
6,"<e>Romney </e>had less to prove, given the la...",108,"[Romney, less, prove, give, last, debate, back..."
7,"<e>Romney </e>- he comes off as a dumbass, but...",160,"[Romney, come, dumbass, love, plan, small, lar..."
8,Great <a>closing</a> by <e>Romney </e>. Summe...,77,"[Great, close, Romney, Summed, well, debate]"
9,Just ready for <e>Romney </e> to become presi...,54,"[ready, Romney, become, president, lol]"
10,"Get him on his trampling on the Constitution, ...",65,"[Get, trample, Constitution, Mitt, Please]"
11,Go <e>Romney</e>!!! I'm tired of all this <a>i...,68,"[Go, Romney, tire, immigration, shit]"
12,<e>Romney</e> is the shit you people are crazy,46,"[Romney, shit, people, crazy]"


In [21]:
df_obama['tweet_clst'] = clean_data(df_obama['tweet'])
df_obama['tweet_clst'] = df_obama['tweet_clst'].apply(lambda x: ','.join(x))
df_obama['tweet_clst'] = df_obama['tweet_clst'].apply(lambda x: x.replace(',' , ' '))
df_obama.head(10)

,tweet,tweet_length,tweet_cleaned,tweet_clst
3,Ditto. I started @247LS 4 years ago. RT @bmorr...,146,"[Ditto, start, years, ago, RT, bmorrissey, wor...",ditto i started 4 years ago rt i work for a sm...
4,I absolutely love <e>Obama</e>'s view in <a>im...,125,"[absolutely, love, Obama, view, immigration, r...",i absolutely love obamas view in immigration h...
5,I'm agreeing completely with <e>Obama</e>'s st...,132,"[agree, completely, Obama, stance, immigration...",im agreeing completely with obamas stance on i...
6,<e>Obama</e>'s <a>smile</a> makes me happy.,43,"[Obama, smile, make, happy]",obamas smile makes me happy
7,Hahahahahaahahha<e> Obama</e>'s <a>rebuttal</a...,83,"[Hahahahahaahahha, Obama, rebuttal, get, actua...",hahahahahaahahha obamas rebuttal got actual cr...
8,If you think the economy has gotten worse duri...,153,"[think, economy, get, worse, Obama, term, plea...",if you think the economy has gotten worse duri...
9,<e>Obama</e>'s <a>debate performance</a> tonig...,91,"[Obama, debate, performance, tonight, better, ...",obamas debate performance tonight about 100000...
10,I like the fact that it's not within <e>Obama<...,76,"[like, fact, within, Obama, comfort, zone, rude]",i like the fact that its not within obamas com...
11,President <e>Obama</e>'s <a>policies</a> HAS N...,149,"[President, Obama, policies, create, problems,...",president obamas policies has not created any ...
12,Loving <e>Obama</e>'s <a>defense of immigratio...,52,"[Loving, Obama, defense, immigration]",loving obamas defense of immigration


In [22]:
df_romney['tweet_clst'] = clean_data(df_romney['tweet'])
df_romney['tweet_clst'] = df_romney['tweet_clst'].apply(lambda x: ','.join(x))
df_romney['tweet_clst'] = df_romney['tweet_clst'].apply(lambda x: x.replace(',' , ' '))
df_romney.head(10)

,tweet,tweet_length,tweet_cleaned,tweet_clst
3,I actually like <e>Romney </e>'s response to ...,58,"[actually, like, Romney, response, immigration]",i actually like romney s response to immigration
4,Just for that <a>immigration statement </a>tha...,115,"[immigration, statement, Romney, answer, enoug...",just for that immigration statement that romne...
5,This man <e>Romney </e>is tearing this dude ...,68,"[man, Romney, tear, dude, economics]",this man romney is tearing this dude up on eco...
6,"<e>Romney </e>had less to prove, given the la...",108,"[Romney, less, prove, give, last, debate, back...",romney had less to prove given the last debate...
7,"<e>Romney </e>- he comes off as a dumbass, but...",160,"[Romney, come, dumbass, love, plan, small, lar...",romney he comes off as a dumbass but i love hi...
8,Great <a>closing</a> by <e>Romney </e>. Summe...,77,"[Great, close, Romney, Summed, well, debate]",great closing by romney summed it all up very ...
9,Just ready for <e>Romney </e> to become presi...,54,"[ready, Romney, become, president, lol]",just ready for romney to become president lol
10,"Get him on his trampling on the Constitution, ...",65,"[Get, trample, Constitution, Mitt, Please]",get him on his trampling on the constitution m...
11,Go <e>Romney</e>!!! I'm tired of all this <a>i...,68,"[Go, Romney, tire, immigration, shit]",go romney im tired of all this immigration shit
12,<e>Romney</e> is the shit you people are crazy,46,"[Romney, shit, people, crazy]",romney is the shit you people are crazy


In [24]:
df_romney.to_csv('cleaned_romney.csv')
df_obama.to_csv('cleaned_obama.csv')

In [ ]:
def text_processing(x):
  new_tweet = clean_sentence(x)
  no_punc_tweet = clean_stopwords(new_tweet)
  return normalization(no_punc_tweet)

pipeline = Pipeline([
    ('bow',CountVectorizer(analyzer=text_processing)),  # strings to token integer counts
    ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
    ('classifier', MultinomialNB()),  # train on TF-IDF vectors w/ Naive Bayes classifier
])

In [ ]:
msg_train, msg_test, label_train, label_test = train_test_split(df_obama['tweet'], df_obama['class'], test_size=0.1)
pipeline.fit(msg_train,label_train)


predictions = pipeline.predict(msg_test)

print(classification_report(predictions,label_test))
print ('\n')
print(confusion_matrix(predictions,label_test))
print(accuracy_score(predictions,label_test))

In [ ]:
pipeline = Pipeline([
    ('bow',CountVectorizer(analyzer=text_processing)),  # strings to token integer counts
    ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
    ('classifier', SGDClassifier()),  # train on TF-IDF vectors w/ Naive Bayes classifier
])

msg_train, msg_test, label_train, label_test = train_test_split(df_obama['tweet'], df_obama['class'], test_size=0.2)
pipeline.fit(msg_train,label_train)


predictions = pipeline.predict(msg_test)

print(classification_report(predictions,label_test))
print ('\n')
print(confusion_matrix(predictions,label_test))
print(accuracy_score(predictions,label_test))